## CRIANDO CONEXÃO

In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Aula").enableHiveSupport().getOrCreate()

In [2]:
spark

## CRIANDO DATAFRAME MANUALMENTE

In [3]:
df_aluno = spark.createDataFrame([
    {"id": 1, "name": "Fabio"},
    {"id": 2, "name": "Joao"},
    {"id": 3, "name": "Fernando"}
])

In [13]:
#tipo do objeto
type(df_aluno)

pyspark.sql.dataframe.DataFrame

## HELP

In [1]:
help(df_aluno)

NameError: name 'df_aluno' is not defined

## CRIANDO DF A PARTIR DE ARQUIVOS

In [14]:
#CSV HADOOP

In [15]:
df_csv = spark.read.csv('/datalake/raw/pessoas',header=True)

In [16]:
#JSON HADOOP

In [17]:
df_json = spark.read.json('/datalake/raw/user')

In [18]:
#JSON S3

In [19]:
df_jsons3 = spark.read.json('s3a://camada-bronze/user/')

In [20]:
df_teste = spark.read.load("/datalake/raw/pessoas",format="csv", sep=",", inferSchema="true", header="true")

In [6]:
#CSV LOCAL
df_local = spark.read.csv('file:///home/user/dados.txt',header=True)

## CRIANDO DF A PARTIR DE BANCO DE DADOS

In [21]:
url = "jdbc:postgresql://postgres:5432/dvdrental"

properties = {
    "user": "admin",
    "password": "admin",
    "driver": "org.postgresql.Driver"
}

In [22]:
df_city = spark.read.jdbc(url=url,table='public.city',properties=properties)
df_county = spark.read.jdbc(url=url,table='public.country',properties=properties)

In [23]:
query = '(select c.city_id ,city, country from public.city c \
        inner join public.country c2 \
        on c2.country_id  = c.country_id) as tab '

In [38]:
df_query = spark.read.jdbc(url=url,table=query,properties=properties)

## AÇÃO E TRANSFORMAÇÃO

In [42]:
#AÇÃO
df_city.show(10)
df_city.count()

+-------+-----------+----------+-------------------+
|city_id|       city|country_id|        last_update|
+-------+-----------+----------+-------------------+
|      2|       Abha|        82|2006-02-15 09:45:25|
|      3|  Abu Dhabi|       101|2006-02-15 09:45:25|
|      4|       Acua|        60|2006-02-15 09:45:25|
|      5|      Adana|        97|2006-02-15 09:45:25|
|      6|Addis Abeba|        31|2006-02-15 09:45:25|
|      7|       Aden|       107|2006-02-15 09:45:25|
|      8|      Adoni|        44|2006-02-15 09:45:25|
|      9| Ahmadnagar|        44|2006-02-15 09:45:25|
|     10|   Akishima|        50|2006-02-15 09:45:25|
|     11|      Akron|       103|2006-02-15 09:45:25|
+-------+-----------+----------+-------------------+
only showing top 10 rows



652

In [50]:
#TRANSFORMAÇÃO
df_city2 = df_city.filter(df_city.country_id == 101).filter(df_city.city.startswith('A'))
df_city2.explain(extended=True)

== Parsed Logical Plan ==
Filter StartsWith(city#4666, A)
+- Filter (cast(country_id#4667 as int) = 101)
   +- Relation [city_id#4665,city#4666,country_id#4667,last_update#4668] JDBCRelation(public.city) [numPartitions=1]

== Analyzed Logical Plan ==
city_id: int, city: string, country_id: smallint, last_update: timestamp
Filter StartsWith(city#4666, A)
+- Filter (cast(country_id#4667 as int) = 101)
   +- Relation [city_id#4665,city#4666,country_id#4667,last_update#4668] JDBCRelation(public.city) [numPartitions=1]

== Optimized Logical Plan ==
Filter ((isnotnull(country_id#4667) AND isnotnull(city#4666)) AND ((country_id#4667 = 101) AND StartsWith(city#4666, A)))
+- Relation [city_id#4665,city#4666,country_id#4667,last_update#4668] JDBCRelation(public.city) [numPartitions=1]

== Physical Plan ==
*(1) Scan JDBCRelation(public.city) [numPartitions=1] [city_id#4665,city#4666,country_id#4667,last_update#4668] PushedFilters: [*IsNotNull(country_id), *IsNotNull(city), *EqualTo(country_id,101

## VALIDANDO DATAFRAME

In [ ]:
#print

In [147]:
df_city.show(10,truncate=False)

+-------+-----------+----------+-------------------+
|city_id|city       |country_id|last_update        |
+-------+-----------+----------+-------------------+
|2      |Abha       |82        |2006-02-15 09:45:25|
|3      |Abu Dhabi  |101       |2006-02-15 09:45:25|
|4      |Acua       |60        |2006-02-15 09:45:25|
|5      |Adana      |97        |2006-02-15 09:45:25|
|6      |Addis Abeba|31        |2006-02-15 09:45:25|
|7      |Aden       |107       |2006-02-15 09:45:25|
|8      |Adoni      |44        |2006-02-15 09:45:25|
|9      |Ahmadnagar |44        |2006-02-15 09:45:25|
|10     |Akishima   |50        |2006-02-15 09:45:25|
|11     |Akron      |103       |2006-02-15 09:45:25|
+-------+-----------+----------+-------------------+
only showing top 10 rows



In [13]:
#describe

In [148]:
df_city.describe().show()

+-------+------------------+--------------------+------------------+
|summary|           city_id|                city|        country_id|
+-------+------------------+--------------------+------------------+
|  count|               652|                 652|               652|
|   mean|326.66257668711654|                null|55.219325153374236|
| stddev|188.62656549041938|                null|31.681053482709448|
|    min|                 1|A Corua (La Corua...|                 1|
|    max|               656|               teste|               111|
+-------+------------------+--------------------+------------------+



In [ ]:
#printSchema

In [149]:
df_city.printSchema()

root
 |-- city_id: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- country_id: short (nullable = true)
 |-- last_update: timestamp (nullable = true)



In [16]:
#count()

In [150]:
df_city.count()

652

In [195]:
#sample([withReplacement, fraction, seed])  Returns a sampled subset of this DataFrame.
df_city.sample(0.1).show()

+-------+--------------------+----------+--------------------+
|city_id|                city|country_id|         last_update|
+-------+--------------------+----------+--------------------+
|    413|      Poos de Caldas|        15| 2006-02-15 09:45:25|
|    236|     Jastrzebie-Zdrj|        76| 2006-02-15 09:45:25|
|    631|                  SP|        15|2023-02-18 11:56:...|
|    224|             Isesaki|        50| 2006-02-15 09:45:25|
|    641|               teste|        15|2023-02-20 11:09:...|
|    189|              Guaruj|        15| 2006-02-15 09:45:25|
|     46|            Baicheng|        23| 2006-02-15 09:45:25|
|     18|Allappuzha (Allep...|        44| 2006-02-15 09:45:25|
|     28| Aparecida de Goinia|        15| 2006-02-15 09:45:25|
|    129|           Cuauhtmoc|        60| 2006-02-15 09:45:25|
|    172|            Fukuyama|        50| 2006-02-15 09:45:25|
|    103|              Carmen|        60| 2006-02-15 09:45:25|
|    612|              TESTE5|       111|2023-02-07 11:

## MANIPULANDO DF

In [151]:
# isEmpty() Returns True if this DataFrame is empty.
df_city.isEmpty()

False

In [152]:
# select(*cols) Projects a set of expressions and returns a new DataFrame.
df_city.select('city').show()

+--------------------+
|                city|
+--------------------+
|                Abha|
|           Abu Dhabi|
|                Acua|
|               Adana|
|         Addis Abeba|
|                Aden|
|               Adoni|
|          Ahmadnagar|
|            Akishima|
|               Akron|
|              al-Ayn|
|           al-Hawiya|
|           al-Manama|
|          al-Qadarif|
|            al-Qatif|
|         Alessandria|
|Allappuzha (Allep...|
|             Allende|
|     Almirante Brown|
|            Alvorada|
+--------------------+
only showing top 20 rows



In [153]:
#orderBy(*cols, **kwargs) Returns a new DataFrame sorted by the specified column(s).
#sort(*cols, **kwargs) Returns a new DataFrame sorted by the specified column(s).

df_city.orderBy('city',ascending=True).show(5)

+-------+--------------------+----------+--------------------+
|city_id|                city|country_id|         last_update|
+-------+--------------------+----------+--------------------+
|      1|A Corua (La Corua...|        87|2023-03-08 13:55:...|
|    656|             ARACAJU|        11|2023-06-07 21:15:...|
|      2|                Abha|        82| 2006-02-15 09:45:25|
|      3|           Abu Dhabi|       101| 2006-02-15 09:45:25|
|      4|                Acua|        60| 2006-02-15 09:45:25|
+-------+--------------------+----------+--------------------+
only showing top 5 rows



In [155]:
from pyspark.sql.functions import desc, asc
df_city.orderBy(df_city.city.asc()).show(5)
df_city.orderBy(df_city.city.desc()).show(5)

+-------+--------------------+----------+--------------------+
|city_id|                city|country_id|         last_update|
+-------+--------------------+----------+--------------------+
|      1|A Corua (La Corua...|        87|2023-03-08 13:55:...|
|    656|             ARACAJU|        11|2023-06-07 21:15:...|
|      2|                Abha|        82| 2006-02-15 09:45:25|
|      3|           Abu Dhabi|       101| 2006-02-15 09:45:25|
|      4|                Acua|        60| 2006-02-15 09:45:25|
+-------+--------------------+----------+--------------------+
only showing top 5 rows

+-------+---------------+----------+--------------------+
|city_id|           city|country_id|         last_update|
+-------+---------------+----------+--------------------+
|    640|          teste|        15|2023-02-20 10:42:...|
|    639|          teste|        15|2023-02-18 12:21:...|
|    641|          teste|        15|2023-02-20 11:09:...|
|    438|s-Hertogenbosch|        67| 2006-02-15 09:45:25|
| 

In [156]:
# where e filter
df_city.where(df_city.city_id == 1).show(truncate=False)

+-------+------------------------+----------+--------------------------+
|city_id|city                    |country_id|last_update               |
+-------+------------------------+----------+--------------------------+
|1      |A Corua (La Corua) TESTE|87        |2023-03-08 13:55:56.748697|
+-------+------------------------+----------+--------------------------+



In [157]:
df_city.filter(df_city.city_id == 1).show(truncate=False)

+-------+------------------------+----------+--------------------------+
|city_id|city                    |country_id|last_update               |
+-------+------------------------+----------+--------------------------+
|1      |A Corua (La Corua) TESTE|87        |2023-03-08 13:55:56.748697|
+-------+------------------------+----------+--------------------------+



In [158]:
#fillna(value[, subset]) Replace null values, alias for na.fill().
df_city.na.fill('').show()

+-------+--------------------+----------+-------------------+
|city_id|                city|country_id|        last_update|
+-------+--------------------+----------+-------------------+
|      2|                Abha|        82|2006-02-15 09:45:25|
|      3|           Abu Dhabi|       101|2006-02-15 09:45:25|
|      4|                Acua|        60|2006-02-15 09:45:25|
|      5|               Adana|        97|2006-02-15 09:45:25|
|      6|         Addis Abeba|        31|2006-02-15 09:45:25|
|      7|                Aden|       107|2006-02-15 09:45:25|
|      8|               Adoni|        44|2006-02-15 09:45:25|
|      9|          Ahmadnagar|        44|2006-02-15 09:45:25|
|     10|            Akishima|        50|2006-02-15 09:45:25|
|     11|               Akron|       103|2006-02-15 09:45:25|
|     12|              al-Ayn|       101|2006-02-15 09:45:25|
|     13|           al-Hawiya|        82|2006-02-15 09:45:25|
|     14|           al-Manama|        11|2006-02-15 09:45:25|
|     15

In [159]:
# distinct() Returns a new DataFrame containing the distinct rows in this DataFrame.
df_city.select('city').distinct().show()

+--------------+
|          city|
+--------------+
|      Fengshan|
|      Chisinau|
|       El Alto|
|          Linz|
|      Myingyan|
|       Udaipur|
|  Juiz de Fora|
|           Oyo|
|       Esfahan|
|        Monywa|
|   Sultanbeyli|
|      Cravihos|
|Dhule (Dhulia)|
|        Jining|
|     Mit Ghamr|
|RIO DE JANEIRO|
|      Salzburg|
|      Sogamoso|
|       Tanauan|
|       Balaiha|
+--------------+
only showing top 20 rows



In [160]:
#first() Returns the first row as a Row.
df_city.first()


Row(city_id=2, city='Abha', country_id=82, last_update=datetime.datetime(2006, 2, 15, 9, 45, 25))

In [162]:
#head([n]) Returns the first n rows.
df_city.head()


Row(city_id=2, city='Abha', country_id=82, last_update=datetime.datetime(2006, 2, 15, 9, 45, 25))

In [163]:
#limit(num) Limits the result count to the number specified.
df_city.limit(5).show()

+-------+-----------+----------+-------------------+
|city_id|       city|country_id|        last_update|
+-------+-----------+----------+-------------------+
|      2|       Abha|        82|2006-02-15 09:45:25|
|      3|  Abu Dhabi|       101|2006-02-15 09:45:25|
|      4|       Acua|        60|2006-02-15 09:45:25|
|      5|      Adana|        97|2006-02-15 09:45:25|
|      6|Addis Abeba|        31|2006-02-15 09:45:25|
+-------+-----------+----------+-------------------+



In [164]:
#tail(num) Returns the last num rows as a list of Row.
df_city.tail(5)


[Row(city_id=651, city='Araraquara', country_id=11, last_update=datetime.datetime(2023, 3, 8, 20, 22, 42, 794627)),
 Row(city_id=652, city='Divinolandia', country_id=11, last_update=datetime.datetime(2023, 3, 8, 20, 28, 30, 59665)),
 Row(city_id=653, city='New York', country_id=11, last_update=datetime.datetime(2023, 3, 8, 20, 28, 33, 380279)),
 Row(city_id=655, city='CAMPINAS', country_id=11, last_update=datetime.datetime(2023, 5, 16, 21, 45, 31, 996979)),
 Row(city_id=656, city='ARACAJU', country_id=11, last_update=datetime.datetime(2023, 6, 7, 21, 15, 45, 6767))]

In [165]:
#take(num) Returns the first num rows as a list of Row.
df_city.take(5)

[Row(city_id=2, city='Abha', country_id=82, last_update=datetime.datetime(2006, 2, 15, 9, 45, 25)),
 Row(city_id=3, city='Abu Dhabi', country_id=101, last_update=datetime.datetime(2006, 2, 15, 9, 45, 25)),
 Row(city_id=4, city='Acua', country_id=60, last_update=datetime.datetime(2006, 2, 15, 9, 45, 25)),
 Row(city_id=5, city='Adana', country_id=97, last_update=datetime.datetime(2006, 2, 15, 9, 45, 25)),
 Row(city_id=6, city='Addis Abeba', country_id=31, last_update=datetime.datetime(2006, 2, 15, 9, 45, 25))]

In [ ]:
# collect() Returns all the records as a list of Row.
df_city.collect()

In [167]:
#groupBy(*cols) Groups the DataFrame using the specified columns, so we can run aggregation on them.
df_city.groupby('country_id').count().show()

+----------+-----+
|country_id|count|
+----------+-----+
|        31|    1|
|        85|   11|
|        65|    1|
|        53|    1|
|        78|    2|
|       108|    2|
|        34|    4|
|       101|    3|
|        81|    1|
|        28|    3|
|        76|    8|
|        27|    3|
|        26|    1|
|        44|   60|
|       103|   35|
|        12|    3|
|        91|    3|
|        22|    3|
|        93|    3|
|       111|   14|
+----------+-----+
only showing top 20 rows



In [168]:
# agg(*exprs) Aggregate on the entire DataFrame without groups (shorthand for df.groupBy().agg()).
df_city \
    .groupby('country_id') \
    .agg({"country_id": "sum"}) \
    .show(5)

+----------+---------------+
|country_id|sum(country_id)|
+----------+---------------+
|        31|             31|
|        85|            935|
|        65|             65|
|        53|             53|
|        78|            156|
+----------+---------------+
only showing top 5 rows



In [170]:
# join inner, cross, outer, full, fullouter, full_outer, left, leftouter, left_outer, right, rightouter, right_outer, semi, leftsemi, left_semi, anti, leftanti and left_anti.
df_city.join(df_county,df_city.country_id == df_county.country_id,'inner').show()

+-------+----------------+----------+-------------------+----------+------------+-------------------+
|city_id|            city|country_id|        last_update|country_id|     country|        last_update|
+-------+----------------+----------+-------------------+----------+------------+-------------------+
|      6|     Addis Abeba|        31|2006-02-15 09:45:25|        31|    Ethiopia|2006-02-15 09:44:00|
|    497|         Springs|        85|2006-02-15 09:45:25|        85|South Africa|2006-02-15 09:44:00|
|    491|      Soshanguve|        85|2006-02-15 09:45:25|        85|South Africa|2006-02-15 09:44:00|
|    437|      Rustenburg|        85|2006-02-15 09:45:25|        85|South Africa|2006-02-15 09:44:00|
|    392|           Paarl|        85|2006-02-15 09:45:25|        85|South Africa|2006-02-15 09:44:00|
|    364|       Newcastle|        85|2006-02-15 09:45:25|        85|South Africa|2006-02-15 09:44:00|
|    273|      Klerksdorp|        85|2006-02-15 09:45:25|        85|South Africa|2

In [171]:
#union
df_city.select('country_id').union(df_county.select('country_id')).show(5)

+----------+
|country_id|
+----------+
|        82|
|       101|
|        60|
|        97|
|        31|
+----------+
only showing top 5 rows



In [172]:
#toDF(*cols) Returns a new DataFrame that with new specified column names
#toJSON([use_unicode])
#toPandas()
df_city.toJSON()
p = df_city.toPandas()
p




/usr/local/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,city_id,city,country_id,last_update
0,2,Abha,82,2006-02-15 09:45:25.000000
1,3,Abu Dhabi,101,2006-02-15 09:45:25.000000
2,4,Acua,60,2006-02-15 09:45:25.000000
3,5,Adana,97,2006-02-15 09:45:25.000000
4,6,Addis Abeba,31,2006-02-15 09:45:25.000000
...,...,...,...,...
647,651,Araraquara,11,2023-03-08 20:22:42.794627
648,652,Divinolandia,11,2023-03-08 20:28:30.059665
649,653,New York,11,2023-03-08 20:28:33.380279
650,655,CAMPINAS,11,2023-05-16 21:45:31.996979


In [173]:
#partitions
df_city.rdd.getNumPartitions()
df_city = df_tab.repartition(2)
df_city.rdd.getNumPartitions()
#df.rdd.partitions.length()
#df.rdd.partitions.size()

2

In [174]:
# drop(*cols) Returns a new DataFrame without specified columns.
df_city.drop('last_update').show()

+-------+-------------------+----------+
|city_id|               city|country_id|
+-------+-------------------+----------+
|    220|             Ilorin|        69|
|    621|             TESTE5|       111|
|    107|         Chandrapur|        44|
|    598|        Zhezqazghan|        51|
|    260|           Kanazawa|        50|
|    321|               Malm|        90|
|    623|           VALINHOS|       111|
|    514|             Tabriz|        46|
|    147|      Dos Quebradas|        24|
|    532|              Tegal|        45|
|    413|     Poos de Caldas|        15|
|    176|        Gandhinagar|        44|
|    487|           Sogamoso|        24|
|    218|             Ikerre|        69|
|    516|             Tafuna|         3|
|    456|Santa Brbara dOeste|        15|
|    564|           Valparai|        44|
|    415|        Probolinggo|        45|
|    398|           Parbhani|        44|
|    265|         Kermanshah|        46|
+-------+-------------------+----------+
only showing top

In [175]:
# dropDuplicates([subset]) Return a new DataFrame with duplicate rows removed, optionally only considering certain columns.
df_city.select('city').count()
df_city.select('city').dropDuplicates().count()

630

In [176]:
# dropna([how, thresh, subset]) Returns a new DataFrame omitting rows with null values.
df_city.na.drop().show()

+-------+---------------+----------+--------------------+
|city_id|           city|country_id|         last_update|
+-------+---------------+----------+--------------------+
|    308|        Lipetsk|        80| 2006-02-15 09:45:25|
|    208|     Hoshiarpur|        44| 2006-02-15 09:45:25|
|    651|     Araraquara|        11|2023-03-08 20:22:...|
|    161|        Escobar|         6| 2006-02-15 09:45:25|
|    586|        Yerevan|         7| 2006-02-15 09:45:25|
|    300|     Lethbridge|        20| 2006-02-15 09:45:25|
|    366|      Nha Trang|       105| 2006-02-15 09:45:25|
|    536|          Tiefa|        23| 2006-02-15 09:45:25|
|    413| Poos de Caldas|        15| 2006-02-15 09:45:25|
|    377|        Okinawa|        50| 2006-02-15 09:45:25|
|    390|            Oyo|        69| 2006-02-15 09:45:25|
|    618|         TESTE5|       111|2023-02-07 12:04:...|
|    467|   Shahr-e Kord|        46| 2006-02-15 09:45:25|
|    363|        NDjamna|        21| 2006-02-15 09:45:25|
|    393|Pachu

In [177]:
#withColumn(colName, col) Returns a new DataFrame by adding a column or replacing the existing column that has the same name.
from pyspark.sql.functions import monotonically_increasing_id
df_city.withColumn('new_id',monotonically_increasing_id()).show(5)

+-------+----------+----------+--------------------+------+
|city_id|      city|country_id|         last_update|new_id|
+-------+----------+----------+--------------------+------+
|    308|   Lipetsk|        80| 2006-02-15 09:45:25|     0|
|    208|Hoshiarpur|        44| 2006-02-15 09:45:25|     1|
|    651|Araraquara|        11|2023-03-08 20:22:...|     2|
|    161|   Escobar|         6| 2006-02-15 09:45:25|     3|
|    586|   Yerevan|         7| 2006-02-15 09:45:25|     4|
+-------+----------+----------+--------------------+------+
only showing top 5 rows



In [178]:
#withColumnRenamed(existing, new) Returns a new DataFrame by renaming an existing column.
df_city.withColumnRenamed('last_update','updated').show()

+-------+---------------+----------+--------------------+
|city_id|           city|country_id|             updated|
+-------+---------------+----------+--------------------+
|    308|        Lipetsk|        80| 2006-02-15 09:45:25|
|    208|     Hoshiarpur|        44| 2006-02-15 09:45:25|
|    651|     Araraquara|        11|2023-03-08 20:22:...|
|    161|        Escobar|         6| 2006-02-15 09:45:25|
|    586|        Yerevan|         7| 2006-02-15 09:45:25|
|    300|     Lethbridge|        20| 2006-02-15 09:45:25|
|    366|      Nha Trang|       105| 2006-02-15 09:45:25|
|    536|          Tiefa|        23| 2006-02-15 09:45:25|
|    413| Poos de Caldas|        15| 2006-02-15 09:45:25|
|    377|        Okinawa|        50| 2006-02-15 09:45:25|
|    390|            Oyo|        69| 2006-02-15 09:45:25|
|    618|         TESTE5|       111|2023-02-07 12:04:...|
|    467|   Shahr-e Kord|        46| 2006-02-15 09:45:25|
|    363|        NDjamna|        21| 2006-02-15 09:45:25|
|    393|Pachu

In [13]:
#alterar typo de coluna
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

df_aluno.withColumn("id",col("id").cast(IntegerType()))
df_aluno.withColumn("id",col("id").cast("int"))
df_aluno.withColumn("id",col("id").cast("integer"))

DataFrame[id: int, name: string]

In [179]:
#foreach(f) Applies the f function to all Row of this DataFrame.
#foreachPartition(f) Applies the f function to each partition of this DataFrame.
def func(df):
    print(df.city)
df_city.foreach(func)

## SQL

In [24]:
#createOrReplaceGlobalTempView(name) 
#createOrReplaceTempView(name)
#createTempView(name)
df_city.createOrReplaceTempView('city')

In [181]:
spark.sql('select * from city limit 5').show()

+-------+----------+----------+--------------------+
|city_id|      city|country_id|         last_update|
+-------+----------+----------+--------------------+
|    308|   Lipetsk|        80| 2006-02-15 09:45:25|
|    208|Hoshiarpur|        44| 2006-02-15 09:45:25|
|    651|Araraquara|        11|2023-03-08 20:22:...|
|    161|   Escobar|         6| 2006-02-15 09:45:25|
|    586|   Yerevan|         7| 2006-02-15 09:45:25|
+-------+----------+----------+--------------------+



## PLANO DE EXECUÇÃO

In [182]:
#explain([extended, mode])Prints the (logical and physical) plans to the console for debugging purposes.
df_city.explain() 

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   ShuffleQueryStage 0
   +- Exchange RoundRobinPartitioning(2), REPARTITION_BY_NUM, [plan_id=2025]
      +- *(1) Scan JDBCRelation(public.city) [numPartitions=1] [city_id#707,city#708,country_id#709,last_update#710] PushedFilters: [], ReadSchema: struct<city_id:int,city:string,country_id:smallint,last_update:timestamp>
+- == Initial Plan ==
   Exchange RoundRobinPartitioning(2), REPARTITION_BY_NUM, [plan_id=2020]
   +- Scan JDBCRelation(public.city) [numPartitions=1] [city_id#707,city#708,country_id#709,last_update#710] PushedFilters: [], ReadSchema: struct<city_id:int,city:string,country_id:smallint,last_update:timestamp>




## TUNNING 

In [ ]:
#HABILITADO DESDE O SPARK 3.2
spark.conf.set("spark.sql.adaptive.enabled", "true")

In [ ]:
# cache unionAll unpersist
df_city.cache()
df_city.persist()
df_city.unpersist()

In [27]:
# HINT
spark.sql('SELECT /*+ REPARTITION(5) */ * FROM city;').explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Exchange RoundRobinPartitioning(5), REPARTITION_BY_NUM, [plan_id=69]
   +- Scan JDBCRelation(public.city) [numPartitions=1] [city_id#133,city#134,country_id#135,last_update#136] PushedFilters: [], ReadSchema: struct<city_id:int,city:string,country_id:smallint,last_update:timestamp>




In [28]:
# BROADCAST VARIABLE
states = {"NY":"New York", "CA":"California", "FL":"Florida"}
broadcastStates = spark.sparkContext.broadcast(states)

## SALVANDO DADOS

In [ ]:
#CSV HADOOP

In [183]:
df_city.write.csv('/datalake/process/city',header=True,sep=',',mode='overwrite')

In [ ]:
#CSV S3

In [184]:
df_city.write.csv('s3a://camada-prata/city/',mode='overwrite')

In [ ]:
#JSON

In [185]:
df_city.write.json('s3a://camada-prata/city_json/',mode='overwrite')

In [ ]:
#PARQUET

In [186]:
df_city.write.parquet('s3a://camada-prata/city_parquet/',mode='overwrite')

In [187]:
spark.sql("SELECT * FROM parquet.`s3a://camada-prata/city_parquet/`").show()

+-------+--------------------+----------+--------------------+
|city_id|                city|country_id|         last_update|
+-------+--------------------+----------+--------------------+
|    373|     Ocumare del Tuy|       104| 2006-02-15 09:45:25|
|    484|            Smolensk|        80| 2006-02-15 09:45:25|
|    418|     Purnea (Purnia)|        44| 2006-02-15 09:45:25|
|    234|            Jamalpur|        12| 2006-02-15 09:45:25|
|    223|                Ipoh|        59| 2006-02-15 09:45:25|
|    217|                 Ife|        69| 2006-02-15 09:45:25|
|     74|            Bhilwara|        44| 2006-02-15 09:45:25|
|    347|              Mwanza|        93| 2006-02-15 09:45:25|
|    294|           Lapu-Lapu|        75| 2006-02-15 09:45:25|
|    329|              Masqat|        71| 2006-02-15 09:45:25|
|    643|      BELO HORIZONTE|        11|2023-03-07 20:50:...|
|    261|         Kanchrapara|        44| 2006-02-15 09:45:25|
|    478|Siliguri (Shiliguri)|        44| 2006-02-15 09

In [ ]:
#ORC

In [188]:
df_city.write.orc('s3a://camada-prata/city_orc/',mode='overwrite')

In [189]:
#DATABASE
df_city.write \
    .jdbc(url=url, table='public.df',properties=properties,mode='overwrite')

## HIVE

In [128]:
df_raw = spark.read.json('s3a://camada-bronze/user/')

In [129]:
df_final = \
df_raw.select( \
     'address.city' \
    ,'address.coordinates.lat' \
    ,'address.coordinates.lng' \
    ,'address.country' \
    ,'address.state' \
    ,'address.street_address' \
    ,'address.street_name' \
    ,'address.zip_code' \
    ,'avatar' \
    ,'credit_card.cc_number' \
    ,'date_of_birth' \
    ,'email' \
    ,'employment.key_skill' \
    ,'employment.title' \
    ,'first_name' \
    ,'gender' \
    ,'id' \
    ,'last_name' \
    ,'password' \
    ,'phone_number' \
    ,'social_insurance_number' \
    ,'subscription.payment_method' \
    ,'subscription.plan' \
    ,'subscription.status' \
    ,'subscription.term' \
    ,'uid' \
    ,'username' \
             )

In [130]:
df_final.count()

82

In [10]:
df_final.write.format('hive').saveAsTable('default.teste3')

In [9]:
spark.sql("show tables from default").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|    teste|      false|
|  default|   teste2|      false|
|  default|     city|      false|
+---------+---------+-----------+



In [139]:
#PARTICIONADO
df.write.partitionBy("country_id").format("parquet").save("/process/tabpart")

## DELTA LAKE

In [1]:
import requests
import json
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("delta").master("spark://spark-master:7077") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.access.key","datalake") \
    .config("spark.hadoop.fs.s3a.secret.key","datalake") \
    .config("spark.hadoop.fs.s3a.endpoint","http://minio:9000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

spark = configure_spark_with_delta_pip(builder).enableHiveSupport().getOrCreate()

In [17]:
def loadData (qtde):
    list = []
    for x in range (qtde):
        print(x)
        r = requests.get('https://random-data-api.com/api/v2/users')
        list.append(r.json())
        req = spark.read.json(spark.sparkContext.parallelize(list))
        req = req.select( \
         'email' \
        ,'first_name' \
        ,'last_name' \
        ,'gender' \
        ,'id' \
        ,'username' \
                 )
    return req

In [18]:
df = loadData(10)

0
1
2
3
4
5
6
7
8
9


In [19]:
df.show(truncate=False)

+-------------------------+----------+----------+-----------+----+---------------+
|email                    |first_name|last_name |gender     |id  |username       |
+-------------------------+----------+----------+-----------+----+---------------+
|thomas.ferry@email.com   |Thomas    |Ferry     |Genderfluid|9258|thomas.ferry   |
|brigette.auer@email.com  |Brigette  |Auer      |Polygender |2479|brigette.auer  |
|johanna.rogahn@email.com |Johanna   |Rogahn    |Genderfluid|8690|johanna.rogahn |
|joannie.russel@email.com |Joannie   |Russel    |Genderqueer|1203|joannie.russel |
|randall.orn@email.com    |Randall   |Orn       |Male       |5704|randall.orn    |
|eddy.monahan@email.com   |Eddy      |Monahan   |Agender    |4722|eddy.monahan   |
|buffy.beahan@email.com   |Buffy     |Beahan    |Non-binary |2661|buffy.beahan   |
|anna.parisian@email.com  |Anna      |Parisian  |Genderqueer|9070|anna.parisian  |
|cecil.gulgowski@email.com|Cecil     |Gulgowski |Non-binary |4545|cecil.gulgowski|
|kur

In [21]:
path = 's3a://camada-prata/user_app'

In [22]:
#salvando como delta table
df.write.format("delta").mode('overwrite').save(path)

In [23]:
deltaTable = DeltaTable.forPath(spark, path)

In [24]:
type(deltaTable)

delta.tables.DeltaTable

In [25]:
d = spark.read.format("delta").load(path)

In [26]:
type(d)

pyspark.sql.dataframe.DataFrame

In [27]:
deltaTable.toDF().show(truncate=False)

+-------------------------+----------+----------+-----------+----+---------------+
|email                    |first_name|last_name |gender     |id  |username       |
+-------------------------+----------+----------+-----------+----+---------------+
|eddy.monahan@email.com   |Eddy      |Monahan   |Agender    |4722|eddy.monahan   |
|buffy.beahan@email.com   |Buffy     |Beahan    |Non-binary |2661|buffy.beahan   |
|anna.parisian@email.com  |Anna      |Parisian  |Genderqueer|9070|anna.parisian  |
|cecil.gulgowski@email.com|Cecil     |Gulgowski |Non-binary |4545|cecil.gulgowski|
|kurt.balistreri@email.com|Kurt      |Balistreri|Male       |6545|kurt.balistreri|
|thomas.ferry@email.com   |Thomas    |Ferry     |Genderfluid|9258|thomas.ferry   |
|brigette.auer@email.com  |Brigette  |Auer      |Polygender |2479|brigette.auer  |
|johanna.rogahn@email.com |Johanna   |Rogahn    |Genderfluid|8690|johanna.rogahn |
|joannie.russel@email.com |Joannie   |Russel    |Genderqueer|1203|joannie.russel |
|ran

In [28]:
### MERGE
#NOVOS DADOS
raw = loadData(10)

0
1
2
3
4
5
6
7
8
9


In [29]:
raw.show(truncate=False)

+-------------------------+----------+---------+-----------+----+---------------+
|email                    |first_name|last_name|gender     |id  |username       |
+-------------------------+----------+---------+-----------+----+---------------+
|yuk.schneider@email.com  |Yuk       |Schneider|Genderfluid|9574|yuk.schneider  |
|claudie.towne@email.com  |Claudie   |Towne    |Bigender   |5756|claudie.towne  |
|wynona.lindgren@email.com|Wynona    |Lindgren |Bigender   |6618|wynona.lindgren|
|daniela.beahan@email.com |Daniela   |Beahan   |Male       |9223|daniela.beahan |
|corazon.jacobi@email.com |Corazon   |Jacobi   |Agender    |8166|corazon.jacobi |
|arlen.jaskolski@email.com|Arlen     |Jaskolski|Non-binary |2016|arlen.jaskolski|
|daryl.ernser@email.com   |Daryl     |Ernser   |Polygender |6468|daryl.ernser   |
|sallie.russel@email.com  |Sallie    |Russel   |Polygender |8647|sallie.russel  |
|blanca.olson@email.com   |Blanca    |Olson    |Agender    |3326|blanca.olson   |
|zack.romaguera@

In [30]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable.alias("process") \
  .merge(
    raw.alias("raw"),
    "process.id = raw.id") \
  .whenNotMatchedInsert(values = {'email' : col('email') \
                                 ,'first_name' : col('first_name') \
                                 ,'last_name' : col('last_name') \
                                 ,'gender' : col('gender') \
                                 ,'id' : col('id') \
                                 ,'username' : col('username') \
                                 }) \
  .whenMatchedUpdate(set = {'email' : col('raw.email') \
                                 ,'first_name' : col('raw.first_name') \
                                 ,'last_name' : col('raw.last_name') \
                                 ,'gender' : col('raw.gender') \
                                 ,'username' : col('raw.username') \
                                 }) \
  .execute()

In [31]:
deltaTable.toDF().show(truncate=False)

+-------------------------+----------+----------+-----------+----+---------------+
|email                    |first_name|last_name |gender     |id  |username       |
+-------------------------+----------+----------+-----------+----+---------------+
|blanca.olson@email.com   |Blanca    |Olson     |Agender    |3326|blanca.olson   |
|arlen.jaskolski@email.com|Arlen     |Jaskolski |Non-binary |2016|arlen.jaskolski|
|sallie.russel@email.com  |Sallie    |Russel    |Polygender |8647|sallie.russel  |
|zack.romaguera@email.com |Zack      |Romaguera |Genderqueer|8631|zack.romaguera |
|daryl.ernser@email.com   |Daryl     |Ernser    |Polygender |6468|daryl.ernser   |
|claudie.towne@email.com  |Claudie   |Towne     |Bigender   |5756|claudie.towne  |
|daniela.beahan@email.com |Daniela   |Beahan    |Male       |9223|daniela.beahan |
|corazon.jacobi@email.com |Corazon   |Jacobi    |Agender    |8166|corazon.jacobi |
|yuk.schneider@email.com  |Yuk       |Schneider |Genderfluid|9574|yuk.schneider  |
|wyn

## SPARK SUBMIT

In [ ]:
from pyspark.sql import SparkSession
import requests
spark = SparkSession.builder.appName("Aula").enableHiveSupport().getOrCreate()
def loadData (qtde):
    list = []
    for x in range (qtde):
        print(x)
        r = requests.get('https://random-data-api.com/api/v2/users')
        list.append(r.json())
        req = spark.read.json(spark.sparkContext.parallelize(list))
        req = req.select( \
         'email' \
        ,'first_name' \
        ,'last_name' \
        ,'gender' \
        ,'id' \
        ,'username' \
                 )
    return req

df = loadData(10)
df.repartition(1).write.parquet('/datalake/raw/api',mode='append')

In [29]:
#spark-submit --master spark://spark-master:7077 --supervise --executor-memory 1G --total-executor-cores 1 programa.py

## STREAMING

In [2]:
from pyspark.sql.types import  StructType, StructField, StringType, LongType, DoubleType, IntegerType, ArrayType
from pyspark.sql.functions import expr, from_json, col, concat
from pyspark.sql import Window
from pyspark.sql import SparkSession
import requests
spark = SparkSession.builder.appName("Aula").enableHiveSupport().getOrCreate()

In [3]:
#API
#https://www.boredapi.com/api/activity/

In [4]:
schema = StructType([
    StructField("activity", StringType()),
    StructField("type", StringType()),
    StructField("participants", IntegerType()),
    StructField("price",DoubleType()),
    StructField("link", StringType()),
    StructField("key", StringType()),
    StructField("accessibility",DoubleType())])
schema

StructType([StructField('activity', StringType(), True), StructField('type', StringType(), True), StructField('participants', IntegerType(), True), StructField('price', DoubleType(), True), StructField('link', StringType(), True), StructField('key', StringType(), True), StructField('accessibility', DoubleType(), True)])

In [5]:
# Create stream dataframe setting kafka server, topic and offset option
df = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "kafka-broker:9092") # kafka server
  .option("subscribe", "atividade") # topic
  .option("startingOffsets", "earliest") # start from beginning 
  #.option("checkpoint","s3a://tmp/checkpoint")    
  .option("kafka.group.id", "spark3")
  .load() 
  )

In [6]:
df.isStreaming

True

In [7]:
# Transform to Output DataFrame
value_df = df.select(from_json(col("value").cast("string"),schema).alias("value"))

In [8]:
exploded_df = value_df.selectExpr('value.activity', 'value.type', 'value.participants', 'value.price',
                                      'value.price','value.link', 'value.key', 'value.accessibility')

In [9]:
exploded_df.printSchema()

root
 |-- activity: string (nullable = true)
 |-- type: string (nullable = true)
 |-- participants: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- price: double (nullable = true)
 |-- link: string (nullable = true)
 |-- key: string (nullable = true)
 |-- accessibility: double (nullable = true)



In [10]:
#ESCREVER EM MEMÓRIA
rawQuery = exploded_df \
        .writeStream \
        .queryName("qraw")\
        .format("memory")\
        .start()

In [11]:
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [12]:
rawQuery.status

{'message': 'Getting offsets from KafkaV2[Subscribe[atividade]]',
 'isDataAvailable': False,
 'isTriggerActive': True}

In [18]:
raw = spark.sql("select * from qraw")


In [22]:
raw.show(truncate=False)

+----------------------------------------------------+------------+------------+-----+-----+----+-------+-------------+
|activity                                            |type        |participants|price|price|link|key    |accessibility|
+----------------------------------------------------+------------+------------+-----+-----+----+-------+-------------+
|Solve a Rubik's cube                                |recreational|1           |0.0  |0.0  |    |4151544|0.1          |
|Go to the gym                                       |recreational|1           |0.2  |0.2  |    |4387026|0.1          |
|Wash your car                                       |busywork    |1           |0.05 |0.05 |    |1017771|0.15         |
|Volunteer at a local animal shelter                 |charity     |1           |0.1  |0.1  |    |1382389|0.5          |
|Make a new friend                                   |social      |1           |0.0  |0.0  |    |1000000|0.0          |
|Go for a run                           

In [25]:
raw.count()

34

In [27]:
raw.select('*').show()

+--------------------+------------+------------+-----+-----+----+-------+-------------+
|            activity|        type|participants|price|price|link|    key|accessibility|
+--------------------+------------+------------+-----+-----+----+-------+-------------+
|Solve a Rubik's cube|recreational|           1|  0.0|  0.0|    |4151544|          0.1|
|       Go to the gym|recreational|           1|  0.2|  0.2|    |4387026|          0.1|
|       Wash your car|    busywork|           1| 0.05| 0.05|    |1017771|         0.15|
|Volunteer at a lo...|     charity|           1|  0.1|  0.1|    |1382389|          0.5|
|   Make a new friend|      social|           1|  0.0|  0.0|    |1000000|          0.0|
|        Go for a run|recreational|           1|  0.0|  0.0|    |6852505|          0.9|
| Write a short story|recreational|           1|  0.0|  0.0|    |6301585|          0.1|
|Fix something tha...|         diy|           1|  0.1|  0.1|    |6925988|          0.3|
|Binge watch a tre...|recreation

In [10]:
#ESCREVER EM DISCO
output_query = exploded_df.writeStream\
        .format("json")\
        .option("path","/datalake/raw/streaming")\
        .option("checkpointLocation", "chck-pnt-dir-kh")\
        .outputMode("append")\
        .queryName("SS Writter")\
        .start()

In [11]:
output_query.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 